In [ ]:
import pandas as pd
from ratdata import data_manager as dm, plot as rdplot
from peewee import JOIN
import scipy.stats
import matplotlib.pyplot as plt

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
peaks = pd.read_excel('plots/blinded_power_spectra/peak_detection_merged.xlsx')
key = pd.read_csv('plots/blinded_power_spectra/key.txt', delimiter=':', header=None)
key.columns = ['number', 'filename']
files = dm.RecordingFile.select(dm.RecordingFile.filename, dm.Rat.full_label, dm.Rat.group, dm.RecordingSlice.recording_rejected)\
    .join(dm.Rat).switch(dm.RecordingFile).join(dm.RecordingSlice, join_type=JOIN.LEFT_OUTER)\
        .where((dm.RecordingFile.condition == 'baseline') & ((dm.RecordingSlice.recording_rejected.is_null()) | (dm.RecordingSlice.recording_rejected == False)))
files_rats = pd.DataFrame(list(files.dicts()))

In [ ]:
peaks_files = peaks[['Number', 'Peak location KO', 'Peak low KO', 'Peak high KO']].merge(key, left_on='Number', right_on='number')
peaks_files_rats = pd.merge(peaks_files, files_rats, on='filename', how='left')
peaks_files_rats['peak_width'] = peaks_files_rats['Peak high KO'] - peaks_files_rats['Peak low KO']

In [ ]:
sham_rats = [r.full_label for r in dm.Rat.select(dm.Rat.full_label).where(dm.Rat.group == 'control').order_by(dm.Rat.full_label)]
ohda_rats = [r.full_label for r in dm.Rat.select(dm.Rat.full_label).where(dm.Rat.group == '6OHDA').order_by(dm.Rat.full_label)]

In [ ]:
print(f'6OHDA mode: {peaks_files_rats[peaks_files_rats["group"] == "6OHDA"]["Peak location KO"].mode()}')
print(f'control mode: {peaks_files_rats[peaks_files_rats["group"] == "control"]["Peak location KO"].mode()}')

In [ ]:
peaks_files_rats[(peaks_files_rats["group"] == "control") & (peaks_files_rats["Peak location KO"] >= 14) & (peaks_files_rats["Peak location KO"] <= 18)].count() / peaks_files_rats[(peaks_files_rats["group"] == "control")].count()

In [ ]:
peaks_files_rats[(peaks_files_rats["group"] == "6OHDA") & (peaks_files_rats["Peak location KO"] >= 14) & (peaks_files_rats["Peak location KO"] <= 18)].count() / peaks_files_rats[(peaks_files_rats["group"] == "6OHDA")].count()

In [ ]:
peak_location_frequency_ohda = 100 * peaks_files_rats[peaks_files_rats["group"] == "6OHDA"]["Peak location KO"].value_counts() / peaks_files_rats[peaks_files_rats["group"] == "6OHDA"]["Peak location KO"].value_counts().sum()

In [ ]:
peak_location_frequency_control = 100 * peaks_files_rats[peaks_files_rats["group"] == "control"]["Peak location KO"].value_counts() / peaks_files_rats[peaks_files_rats["group"] == "control"]["Peak location KO"].value_counts().sum()


In [ ]:
fig = plt.figure(figsize=(21, 9))
plt.bar(peak_location_frequency_ohda.index.tolist(), peak_location_frequency_ohda, facecolor=rdplot.sham_ohda_palette["ohda"], alpha=0.7)
plt.bar(peak_location_frequency_control.index.tolist(), peak_location_frequency_control, facecolor=rdplot.sham_ohda_palette["sham"], alpha=0.5)
ax = fig.axes[0]
plt.title("Density of most significant peak locations in sham and 6-OHDA rats", fontsize=30)
plt.legend(["6-OHDA", "sham"], fontsize=30)
ax.tick_params(axis='both', which='major', labelsize=22)
plt.xlabel('Frequency [Hz]', fontsize=22)
plt.ylabel('Percent of total', fontsize=22)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.savefig('plots/peaks/0_peak_location_density.png', bbox_inches='tight')
plt.savefig('plots/peaks/0_peak_location_density.svg', bbox_inches='tight')

In [ ]:
print(f"OHDA kurtosis: {peak_location_frequency_ohda.kurtosis()}, control kurtosis: {peak_location_frequency_control.kurtosis()}")
scipy.stats.ks_2samp(peak_location_frequency_control, peak_location_frequency_ohda)

In [ ]:
rdplot.plot_peak_info(peaks_files_rats, 'Peak location KO', ohda_rats, sham_rats[1:], 'plots/blinded_power_spectra/0_peak_locations_baseline', ylim=[12.7, 29.3], figsize=(18, 10))

In [ ]:
rdplot.plot_peak_info(peaks_files_rats, 'peak_width', ohda_rats, sham_rats[1:], 'plots/blinded_power_spectra/0_peak_widths_baseline', ylim=[0.7, 8.3], figsize=(18, 10))